In [2]:
import urllib
from urllib.request import Request, urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup as bs
import pandas as pd
from math import nan
from datetime import date
import psycopg2
import time
import requests
import gzip
import math

c:\Users\Clamfighter\Machine_Learning_Project\my_env\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
def getHTMLforPage(url):
    print('retrieving HTML for page: ',url)
    req = Request(url)
    print(req)
    try:
        html_page = gzip.decompress(urlopen(req).read()).decode('utf-8')
    except OSError:
        html_page = urlopen(req).read().decode('utf-8')

    return bs(html_page, 'html.parser')

In [2]:
url = 'https://www.superdrug.com/health/vitamins-supplements/vitamin-d/abidec-multivitamin-drops-for-babies-children-25ml/p/199810'
r = requests.get(url)
r.text


In [2]:
import urllib.request

fp = urllib.request.urlopen("https://www.superdrug.com/health/vitamins-supplements/vitamin-d/abidec-multivitamin-drops-for-babies-children-25ml/p/199810")
mybytes = fp.read()

mystr = mybytes.decode("utf8")
fp.close()

print(mystr)

In [16]:
def getBrandURls():
    print('Getting brands page URLs')
    brandsSoup = getHTMLforPage('https://www.boots.com/brands')
    brandListSelectors=brandsSoup.find_all("div", {"class": "brand_list_selector"})
    brandListSelectors=brandListSelectors[0].find_all('a', href=True)
    brandListSelectors=[e['href'] for e in brandListSelectors]

    brandsURLsComplete=[]
    print('getting brand URLs from page')
    #brandListSelectors=['https://www.boots.com/brands']
    for brandsPageURL in brandListSelectors:
        brandsSoup = getHTMLforPage(brandsPageURL)
        brandsURls = brandsSoup.find_all("div", {"id": "brand_list_viewer"})
        brandsURls=brandsURls[1].find_all('a', href=True)
        brandsURls=[e['href'] for e in brandsURls]
        for brandURL in brandsURls:
            brandsURLsComplete.append(brandURL)
    
    return brandsURLsComplete


In [30]:
def getProductURLsforPage(html):
    productURLs = html.find_all("a", {"class": "product_name_link product_view_gtm"})
    productURLs=[e['href'] for e in productURLs]
    print(productURLs)
    return productURLs

In [ ]:
brandURL='https://www.boots.com/clinique/clinique-full-range#facet:-7000000000000135605102979910132109111105115116117114105115101114&productBeginIndex:0&orderBy:&pageView:grid&minPrice:&maxPrice:&pageSize:&'

def getProductURLsForBrand(brandURL):
    print('Getting Brand URL')
    #brandURL=(brandURL+'#facet:&productBeginIndex:0&orderBy:&pageView:grid&minPrice:&maxPrice:&pageSize:180&')
    print(brandURL)
    brandPageHTML = getHTMLforPage(brandURL)
    #TODO deal with multiple pages on brand side
    
    '''
    Checks if its actually the product page, or a fall page.
    If theres a button to go to the actual products page, it gets that link
    '''
    shopAllURL = brandPageHTML.find_all("div", {"class": "oct-aem-button"})
    if shopAllURL !=[]:
        print('forwarding to product page')
        shopAllURL= shopAllURL[0].find_all('a', href=True)
        shopAllURL = shopAllURL[0]['href']
        brandPageHTML = getHTMLforPage(shopAllURL)


    '''
    Checking # of product on page
    '''


    numberOfProducts=brandPageHTML.find_all("span", {"class": "showing_products_total"})[0].text.strip()
    if (int(numberOfProducts)<=180):
        productURLs = getProductURLsforPage(brandPageHTML)

    else:
        print(numberOfProducts)
        productURLs = getProductURLsforPage(brandPageHTML)
        numOfPages = math.ceil(int(numberOfProducts)/180)
        url = brandURL[:-58]
        for i in range(1,numOfPages):
            j=numOfPages*180
            url=(url+'0&orderBy:&pageView:grid&minPrice:&maxPrice:&pageSize:180&')
            productURLs.extend(getProductURLsforPage(brandPageHTML))


        
    
    return productURLs
    
getProductURLsForBrand(brandURL)


In [ ]:
import csv

with open('F:\Data Mining\Boots\productLinks.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    for i in range(0,1000):
        productNumber=str(productNumber).zfill(6)
        url='https://html.duckduckgo.com/html/?q=10'+productNumber+'+site%3Aboots.com'


        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

        html_page = urlopen(req).read().decode('utf-8')
        soup = bs(html_page, 'html.parser')
        links = soup.find_all("h2", {"class": "result__title"})
        links = links[0].find_all('a', href=True)
        if links:
            print(links[0]['href'])
            row=[productNumber,links[0]['href']]
            writer.writerow(row)

In [1]:
from autoscraper import AutoScraper

url = 'https://stackoverflow.com/questions/2081586/web-scraping-with-python'

scraper = AutoScraper()

scraper.get_result_exact('https://www.boots.com/clinique/clinique-full-range#facet:&productBeginIndex:0&orderBy:&pageView:grid&minPrice:&maxPrice:&pageSize:&')

[]

In [51]:
import os, sys
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time

Get Data from product Link

In [10]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized");
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

link = r'https://www.superdrug.com/skin/oilatum/b/414003'

In [11]:
chromedriver_path = 'C:\webdrivers\chromedriver.exe'

if getattr(sys, 'frozen', False): 
            #executed as a bundled exe, the driver is in the extracted folder
            browser = webdriver.Chrome(chromedriver_path,options=options)
else:
    # executed as a simple script, the driver should be in `PATH`
    browser = webdriver.Chrome(options=options)

browser.get(link)

In [9]:
nameContainer = browser.find_element(By.CLASS_NAME, "product-details-title__container")

name = nameContainer.text
print(name)


metatags = nameContainer.find_element("xpath","//meta[@itemprop='sku']")
print(metatags.get_attribute("content"))


I Heart Revolution Tasty Avocado Brow Gel Clear
791793


In [10]:
promotion = browser.find_element(By.CLASS_NAME, "product-description__promotions").text

print(promotion)

In [11]:
price = browser.find_element(By.CLASS_NAME, "price__current").text
print(price)

£5.00


In [12]:
try:
    pricePerUnit = browser.find_element(By.CLASS_NAME, "price__per-unit").text
except NoSuchElementException:
    pricePerUnit = 'unknown'
print(pricePerUnit)

unknown


Get List of product numbers

In [77]:
chromedriver_path = 'C:\webdrivers\chromedriver.exe'
link = r'https://www.superdrug.com/a-z-brands'
if getattr(sys, 'frozen', False): 
            #executed as a bundled exe, the driver is in the extracted folder
            browser = webdriver.Chrome(chromedriver_path,options=options)
else:
    # executed as a simple script, the driver should be in `PATH`
    browser = webdriver.Chrome(options=options)

browser.get(link)

862
862


In [45]:
browser.get('https://www.superdrug.com/skin/dove/b/403024?pageSize=10000000')

In [6]:
brandlinks = ['https://www.superdrug.com/skin/dove/b/403024']

In [94]:
def getProductDataForPage():
    productsData=[]

    productElems = browser.find_elements(By.CLASS_NAME, 'product-list-item')

    for productElem in productElems:
        #find product name
        try:
            productName=''
            productName=productElem.find_element(By.CLASS_NAME, 'cx-product-name').text
        except NoSuchElementException:
            productName='NoSuchElementException'
        #find product price
        try:
            productPrice=''
            productPrice=productElem.find_element(By.CLASS_NAME, 'price__current').text
        except NoSuchElementException:
            productName='NoSuchElementException'
        #get product link
        try:
            imageLinkElem=productElem.find_element(By.CLASS_NAME, 'product-image-container')
            productLink=imageLinkElem.get_attribute("href")
            #use link to get SKU
            SKU=productLink.split('/')[-1]
        except NoSuchElementException:
            SKU='NoSuchElementException'
            productLink='NoSuchElementException'

        #get promotion text & link to promotion
        ProductPromotions=[]
        try:
            allPromotionelem=productElem.find_element(By.CLASS_NAME, 'product-list-item__promotions')
            promotionelems = allPromotionelem.find_elements("xpath",(".//*"))
            promotionLinks=[]
            for elem in promotionelems:
                link=elem.get_attribute("href")
                if link != None:
                    promotionLinks.append(link)
                    ProductPromotions.append(link.split('/')[-1].replace("-", " " ))
        except NoSuchElementException:
            ProductPromotions=[]
            promotionLinks=[]
       




        productData=[SKU,productName,productPrice,productLink,ProductPromotions,promotionLinks]

        productsData.append(productData)
    return productsData



In [87]:
def getProductsforBrand(brandlink):

    expandedLink = brandlink+'?pageSize=10000000'
    browser.get(expandedLink)
    time.sleep(1)






    productData=getProductDataForPage()#get data for current page
    pageselems = browser.find_elements(By.CLASS_NAME, 'last')

    #get the number of pages of products
    if len(pageselems) ==1: #if theres a last page elem, this is the number of pages
        numpages=int(pageselems[0].text)
    elif len(pageselems) ==0:#no page elem if only 1 page, otherwise # of elem = number of pages
        pageselems = browser.find_elements(By.CLASS_NAME, 'page')
        if len(pageselems) >0:
            numpages = len(pageselems)#find number of pages
            
        else:
            numpages = 1
    #print('Number of Pages: ',str(numpages))
    if numpages>1:
        for i in range(1,numpages):
            #print(i)
            browser.get(expandedLink+'&currentPage='+str(i))
            time.sleep(1)
            currPageProducts = getProductDataForPage()
            productData.extend(currPageProducts)
            #print('done page: ',i)

        brandProductsTuple=[]
    
    return productData
    

In [95]:
chromedriver_path = 'C:\webdrivers\chromedriver.exe'
link = r'https://www.superdrug.com/a-z-brands'
if getattr(sys, 'frozen', False): 
            #executed as a bundled exe, the driver is in the extracted folder
            browser = webdriver.Chrome(chromedriver_path,options=options)
else:
    # executed as a simple script, the driver should be in `PATH`
    browser = webdriver.Chrome(options=options)

browser.get(link)
time.sleep(4)

brandelems = browser.find_elements(By.CLASS_NAME, 'az-brands__list-letter-group-brand-item-link')
print(len(brandelems))
brandProducts = []
brandlinks=[]
brandNames=[]
for elem in brandelems:
    brandName = elem.text
    brandNames.append(brandName)
    brandlinks.append(elem.get_attribute("href"))
for i in range(len(brandlinks)):
    brandLink=brandlinks[i]
    brandName = brandNames[i]
    print(brandName)
    products = getProductsforBrand(brandLink)
    brandProducts.append([brandName,products])


862
Abidec
ab masks
Acnecide
Acriflex
Actifed
ActiVeman
Active Woman
Acuvue
Advantage
Afrocenchix
After Bite
Aguulp
AHC
AHS
Aidapt
Alcon
Alflorex
Alka-Seltzer
Allevia
ALLSAINTS
All Tigers
Almased
Almus
Aloclair
Aloe Pura
Alpecin
Always
Always Discreet
ALYA
Amplex
Anadin
Anbesol
Andrews
Anna Sui
Anthisan
Anusol
Aptamil
Aquafresh
Aqua Vape
Aqua Wipes
Ardell
Ariana Grande
Armani
Arm & Hammer
Aroma Works
Artelac
Arthr
Artiscent
Ashton and Parsons
AS I AM
Astral
Atrixo
Aussie
Australian Glow
Aveeno
Aveeno Baby
Aveeno Hair
Avene
A Vogel
B.
Baby Dove
Babyliss
Baby Shark
Bach
Badedas
Balance Activ
Banana Republic
Bare Feet
Barry M
Bassetts
Batiste
Batman
Bausch & Lomb
Bayleys Of Bond St
Baylis & Harding
Bazuka
B. Cosmetics
Bear Fruits
Beauty Laundrette
Beauty Pro
Beckham
BecoDefense
Beconase
Beechams
Bellamianta
Bell Sons & Co
Benacort
Benadryl
Benylin
Bepanthen
Berocca
Bespoke
BetterYou
Beverly Hills
Beyond NRG
Be You
Bic
Bickiepegs
BILLIE EILISH
Billion Dollar Smile
BIODERMA
BioFreeze
Biogla

In [115]:
productsOutput=[]

for brandProduct in brandProducts:
    brand = brandProduct[0]
    products = brandProduct[1]
    print(brand)
    for product in products:
        productOut=product[:4]#add SKU, name, price & link to outarray for product
        productOut.append(brand)
        #collect promotions and promotion links in 2 arrays
        promotions = product[4]
        promotionsURLs = product[5]
        for i in range(0,len(promotions)):
            productOut.extend([promotions[i],promotionsURLs[i]])
        productsOutput.append(productOut)

productsOutput


Abidec
ab masks
Acnecide
Acriflex
Actifed
ActiVeman
Active Woman
Acuvue
Advantage
Afrocenchix
After Bite
Aguulp
AHC
AHS
Aidapt
Alcon
Alflorex
Alka-Seltzer
Allevia
ALLSAINTS
All Tigers
Almased
Almus
Aloclair
Aloe Pura
Alpecin
Always
Always Discreet
ALYA
Amplex
Anadin
Anbesol
Andrews
Anna Sui
Anthisan
Anusol
Aptamil
Aquafresh
Aqua Vape
Aqua Wipes
Ardell
Ariana Grande
Armani
Arm & Hammer
Aroma Works
Artelac
Arthr
Artiscent
Ashton and Parsons
AS I AM
Astral
Atrixo
Aussie
Australian Glow
Aveeno
Aveeno Baby
Aveeno Hair
Avene
A Vogel
B.
Baby Dove
Babyliss
Baby Shark
Bach
Badedas
Balance Activ
Banana Republic
Bare Feet
Barry M
Bassetts
Batiste
Batman
Bausch & Lomb
Bayleys Of Bond St
Baylis & Harding
Bazuka
B. Cosmetics
Bear Fruits
Beauty Laundrette
Beauty Pro
Beckham
BecoDefense
Beconase
Beechams
Bellamianta
Bell Sons & Co
Benacort
Benadryl
Benylin
Bepanthen
Berocca
Bespoke
BetterYou
Beverly Hills
Beyond NRG
Be You
Bic
Bickiepegs
BILLIE EILISH
Billion Dollar Smile
BIODERMA
BioFreeze
Bioglan
Bi

[['199810',
  'Abidec Multivitamin Drops for Babies & Children 25ml',
  '£6.49',
  'https://www.superdrug.com/health/vitamins-supplements/vitamin-d/abidec-multivitamin-drops-for-babies-children-25ml/p/199810',
  'Abidec',
  '3 for 2 mix and match',
  'https://www.superdrug.com/a/87600/3-for-2-mix-and-match'],
 ['736742',
  'Abidec Advanced Multivitamin Syrup Plus Omega 6 & 9 150ml',
  '£4.75',
  'https://www.superdrug.com/health/vitamins-supplements/childrens-vitamins/abidec-advanced-multivitamin-syrup-plus-omega-6-9-150ml/p/736742',
  'Abidec',
  '3 for 2 mix and match',
  'https://www.superdrug.com/a/87600/3-for-2-mix-and-match'],
 ['798671',
  'Abidec Immune Support Drops',
  '£14.99',
  'https://www.superdrug.com/health/vitamins-supplements/childrens-vitamins/abidec-immune-support-drops/p/798671',
  'Abidec',
  '3 for 2 mix and match',
  'https://www.superdrug.com/a/87600/3-for-2-mix-and-match'],
 ['798166',
  'The ab Face Mask Replacement Filters',
  '£4.99',
  'https://www.superd

In [117]:
import pandas as pd
df = pd.DataFrame(productsOutput)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,199810,Abidec Multivitamin Drops for Babies & Childre...,£6.49,https://www.superdrug.com/health/vitamins-supp...,Abidec,3 for 2 mix and match,https://www.superdrug.com/a/87600/3-for-2-mix-...,None,None,None,None,None,None,None,None
1,736742,Abidec Advanced Multivitamin Syrup Plus Omega ...,£4.75,https://www.superdrug.com/health/vitamins-supp...,Abidec,3 for 2 mix and match,https://www.superdrug.com/a/87600/3-for-2-mix-...,None,None,None,None,None,None,None,None
2,798671,Abidec Immune Support Drops,£14.99,https://www.superdrug.com/health/vitamins-supp...,Abidec,3 for 2 mix and match,https://www.superdrug.com/a/87600/3-for-2-mix-...,None,None,None,None,None,None,None,None
3,798166,The ab Face Mask Replacement Filters,£4.99,https://www.superdrug.com/health/first-aid/fac...,ab masks,None,None,None,None,None,None,None,None,None,None
4,798162,Reusable Antibacterial Ab Face Mask,£9.99,https://www.superdrug.com/health/first-aid/fac...,ab masks,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170,766962,10 Motives Menthol Refills 16mg,£7.99,https://www.superdrug.com/health/smoking-contr...,10 Motives,None,None,None,None,None,None,None,None,None,None
14171,766940,10 Motives Disposable Regular 18mg,£6.99,https://www.superdrug.com/health/smoking-contr...,10 Motives,None,None,None,None,None,None,None,None,None,None
14172,766954,10 Motives Rechargable Starter Kit 16mg,£9.99,https://www.superdrug.com/health/smoking-contr...,10 Motives,None,None,None,None,None,None,None,None,None,None
14173,11444,4 Head Effective Headache Relief,£4.79,https://www.superdrug.com/health/pain-relief/p...,4 Head,None,None,None,None,None,None,None,None,None,None


In [119]:
df.to_csv('testOutput.csv',encoding='utf-8')